# Sistema de predicción de ventas
Queremos establecer el almacén de nuestra empresa en otra localización y necesitamos estimar el ritmo de las ventas, que desde la creación de la empresa ha ido en aumento, para los próximos meses, a fin de proveer el espacio que necesitaremos.

In [15]:
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_absolute_error
from prophet import Prophet
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from tensorboard import summary

import warnings
warnings.filterwarnings('ignore')

In [16]:
df_ini = pd.read_csv('../data/raw/10 ventas.csv', sep=',')
df_ini.head()

,date,sales
0,2022-09-03 17:10:08.079328,55.292157
1,2022-09-04 17:10:08.079328,53.803211
2,2022-09-05 17:10:08.079328,58.141693
3,2022-09-06 17:10:08.079328,64.530899
4,2022-09-07 17:10:08.079328,66.013633


In [17]:
df_ini.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    366 non-null    object 
 1   sales   366 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.8+ KB


In [18]:
# Elimino duplicados de las columnas restantes.

df= df_ini.drop_duplicates()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    366 non-null    object 
 1   sales   366 non-null    float64
dtypes: float64(1), object(1)
memory usage: 5.8+ KB


In [19]:
df.isna().sum()


date     0
sales    0
dtype: int64

In [20]:
df.nunique()

date     366
sales    366
dtype: int64

## Data Cleaning

- Analizamos 'date' y 'sales'

In [21]:
df['date'].value_counts()

date
2023-09-03 17:10:08.079328    1
2022-09-03 17:10:08.079328    1
2022-09-04 17:10:08.079328    1
2022-09-05 17:10:08.079328    1
2022-09-06 17:10:08.079328    1
                             ..
2022-09-20 17:10:08.079328    1
2022-09-21 17:10:08.079328    1
2022-09-22 17:10:08.079328    1
2022-09-23 17:10:08.079328    1
2022-09-24 17:10:08.079328    1
Name: count, Length: 366, dtype: int64

In [22]:
df['sales'].value_counts()

sales
1000.482785    1
55.292157      1
53.803211      1
58.141693      1
64.530899      1
              ..
93.631101      1
97.788518      1
96.889768      1
94.395825      1
106.618390     1
Name: count, Length: 366, dtype: int64

Paso 2: Construye y analiza la serie temporal
Construye la estructura de datos válida para la serie temporal, grafícala y, a continuación, analízala y responde a las siguientes preguntas:

¿Cuál es el tensor de la serie temporal?
¿Cuál es la tendencia?
¿Es estacionaria?
¿Existe variabilidad o presencia de ruido?
Nota: Un tensor en una serie temporal es la unidad de tiempo mínima para la cual hay datos. Puede ser cada segundo, minuto, hora, día, semana, mes...

Paso 3: Entrena un ARIMA
Utiliza los datos de entrenamiento para encontrar la mejor parametrización de tu modelo ARIMA.

Paso 4: Predice con el conjunto de test
Ahora utiliza el modelo entrenado con el conjunto de prueba y compara los puntos con los reales. Mide el rendimiento de la serie temporal.

Paso 5: Guarda el modelo
Almacena el modelo en la carpeta correspondiente.
